<a href="https://colab.research.google.com/github/linyuehzzz/5523_project/blob/main/sgd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**Stochastic Gradient Descent for Logistic Regression**
This code implements and tests the SGD algorithm for logistic regression
in different scenarios.  
Yue Lin (lin.3326 at osu.edu)  
Created: 11/12/2020

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

#### **Set up libraries**

In [14]:
import numpy as np
import random
import math
from sklearn.linear_model import SGDClassifier
from sklearn.preprocessing import StandardScaler

#### **Prepare data**

##### Generate training data

In [24]:
def train_data(n_epoch, train_n):
  train_x = np.random.uniform(-1, 1, (train_n * n_epoch, 4))
  b = np.ones((train_n * n_epoch, 1))
  np.append(train_x, b, axis=1)
  train_y = np.array([random.randrange(-1, 2, 2) for i in range(train_n * n_epoch)])
  # print(train_x, train_y)
  return train_x, train_y

##### Generate test data

In [21]:
def test_data(test_n):
  test_x = np.random.uniform(-1, 1, (test_n, 4))
  b = np.ones((test_n, 1))
  np.append(test_x, b, axis=1)
  test_y = np.array([random.randrange(-1, 2, 2) for i in range(test_n)])
  # print(test_x, test_y)
  return test_x, test_y

#### **Train**
https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html

https://machinelearningmastery.com/implement-logistic-regression-stochastic-gradient-descent-scratch-python/

##### Predict using logistic regression

In [ ]:
# Make a prediction with coefficients
def pred(row, w):
	yhat = w[0]
	for i in range(len(row)-1):
		yhat += w[i + 1] * row[i]
	return 1.0 / (1.0 + exp(-yhat))

##### Estimate weight vector using SGD

In [34]:
def sgd(train, l_rate, n_epoch, bs):
  w = np.zeros((1, len(train_x[0])))
  for epoch in range(n_epoch):
    l_ave = l_min = l_var = 0.
    err_ave = err_var = 0.
    for row in train[epoch * bs: (epoch + 1) * bs, :]:
      yhat = pred(row, w)
      error = row[-1] - yhat
      sum_error += error**2
      w[0] = w[0] + l_rate * error * yhat * (1.0 - yhat)
      for i in range(len(row)-1):
        w[i + 1] = w[i + 1] + l_rate * error * yhat * (1.0 - yhat) * row[i]
    print('>epoch=%d, lrate=%.3f, error=%.3f' % (epoch, l_rate, sum_error))
  return w

#### **Test**
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.log_loss.html

In [ ]:
>>> clf.fit(X, Y)
Pipeline(steps=[('standardscaler', StandardScaler()),
                ('sgdclassifier', SGDClassifier())])
>>> print(clf.predict([[-0.8, -1]]))

#### **Wrapper**

In [22]:
# Generate test data
test_n = 400
test_x, test_y = test_data(test_n)

# Generate training data
n_epoch = 30
train_n = 50
train_x, train_y = train_data(n_epoch, train_n)
